In [2]:
import pandas as pd
import numpy as np 

countryName = []
eprtrSectorName = []
EPRTRAnnexIMainActivityLabel = []
FacilityInspireID = []
facilityName = []
City =[]
targetRelease = []
pollutant = []
reportingYear = []
MONTH= []
DAY = []
CONTINENT= []
max_wind_speed =[]
avg_wind_speed= []
min_wind_speed = []
max_temp = []
avg_temp= []
min_temp=[]
FOGDAYS=[]
CITYID = []


url1 = 'http://schneiderapihack-env.eba-3ais9akk.us-east-2.elasticbeanstalk.com/first'
url2 = 'http://schneiderapihack-env.eba-3ais9akk.us-east-2.elasticbeanstalk.com/second'
url3 = 'http://schneiderapihack-env.eba-3ais9akk.us-east-2.elasticbeanstalk.com/third'
urls = [url1,url2,url3]
for url in urls: 

    from urllib.request import urlopen

    # import json
    import json
    # store the URL in url as 
    # parameter for urlopen
    url = url

    # store the response of URL
    response = urlopen(url)

    # storing the JSON response 
    # from url in data
    data_json = json.loads(response.read())

    
    for jsons in data_json:
        countryName.append(jsons.get('countryName'))
        eprtrSectorName.append(jsons.get('eprtrSectorName'))
        EPRTRAnnexIMainActivityLabel.append(jsons.get('EPRTRAnnexIMainActivityLabel'))
        FacilityInspireID.append(jsons.get('FacilityInspireID'))
        facilityName.append(jsons.get('facilityName'))
        City.append(jsons.get('City'))
        targetRelease.append(jsons.get('targetRelease'))
        pollutant.append(jsons.get('pollutant'))
        reportingYear.append(jsons.get('reportingYear'))
        MONTH.append(jsons.get('MONTH'))
        DAY.append(jsons.get('DAY'))
        CONTINENT.append(jsons.get('CONTINENT'))
        max_wind_speed.append(jsons.get('max_wind_speed'))
        avg_wind_speed.append(jsons.get('avg_wind_speed'))
        min_wind_speed.append(jsons.get('min_wind_speed'))
        max_temp.append(jsons.get('max_temp'))
        avg_temp.append(jsons.get('avg_temp'))
        min_temp.append(jsons.get('min_temp'))
        FOGDAYS.append(jsons.get('DAY WITH FOGS'))
        CITYID.append(jsons.get('CITY ID'))

    
df_json = pd.DataFrame({'countryName':countryName,
                        'eprtrSectorName':eprtrSectorName,
                        'EPRTRAnnexIMainActivityLabel':EPRTRAnnexIMainActivityLabel,
                        'FacilityInspireID':FacilityInspireID,
                        'facilityName':facilityName,
                        'City':City,
                        'targetRelease':targetRelease,
                        'pollutant':pollutant,
                        'reportingYear':reportingYear,
                        'MONTH':MONTH,
                        'DAY':DAY,
                        'CONTINENT':CONTINENT,
                        'max_wind_speed':max_wind_speed,
                        'avg_wind_speed':avg_wind_speed,
                        'min_wind_speed':min_wind_speed,
                        'max_temp':max_temp,
                        'avg_temp':avg_temp,
                        'min_temp': min_temp,
                        'FOGDAYS':FOGDAYS,
                        'CITYID':CITYID})

In [3]:
df_json.rename(columns = {'FOGDAYS':'DAY WITH FOGS'}, inplace = True)

In [148]:
df1 = pd.read_csv(r"C:\Users\Usuario\Desktop\SchneiderHackaton\train1.csv")
df2 = pd.read_csv(r"C:\Users\Usuario\Desktop\SchneiderHackaton\train2.csv",sep=';')

df_csv = pd.concat([df1,df2])
df_total = pd.concat([df_csv,df_json])
df_total.reset_index().drop('index',inplace =True,axis = 1)



In [5]:
df_total['pollutant'].unique()

array(['Carbon dioxide (CO2)', 'Nitrogen oxides (NOX)', 'Methane (CH4)'],
      dtype=object)

In [149]:
dict_map ={'Carbon dioxide (CO2)':1,'Nitrogen oxides (NOX)':0,'Methane (CH4)':2}

df_total['pollutant'] = df_total['pollutant'].map(dict_map)

for columns in df_total.columns:
    try:
        df_total[columns]= df_total[columns].astype(float)
    except:
        'None'

In [7]:
#mas o menos de 5 dias de niebla
def foggy(x):
    if x >5:
        return 1
    else:
        return 0 
    
df_total['FOGGY']= df_total['DAY WITH FOGS'].apply(foggy)
#Es o no es el reino unido, donde parece que tiene una distribucion de pollutant muy distinta a los otros paises
def isuk(x):
    if x == 'United Kingdom':
        return 1
    else:
        return 0 
df_total['ISUK'] = df_total['countryName'].apply(isuk)

In [153]:
df_proc = df_total.drop(columns = ['countryName','FacilityInspireID','facilityName','City','targetRelease','CONTINENT','eprtrSectorName'])
df_proc = df_proc.drop(columns = ['CITY ID','REPORTER NAME','CITYID'])


df_proc.drop(columns = ['DAY WITH FOGS','reportingYear','MONTH','DAY'],inplace = True)


In [167]:
dftest.drop(columns = ['EPRTRSectorCode','EPRTRAnnexIMainActivityCode','reportingYear','MONTH','DAY','CITY ID','REPORTER NAME','DAY WITH FOGS'],inplace = True)

In [194]:
from sklearn.model_selection import train_test_split 


X = df_dummed[df_dummed.columns[1:]]
y = df_dummed['pollutant']
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.15,
                                                    random_state=2)
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=250)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.6815642458100558


In [23]:
v= df_dummed["avg_wind_speed"]
t= df_dummed["avg_temp"]
df_dummed["Temp_Sen"] = 13.12 + 0.6215 * t - 11.73 * v ** 0.16 + 0.3965 * t * v ** 0.16

In [28]:
#Desviacion temp maxima
df_dummed['var_max_temp'] = df_dummed.max_temp - df_dummed.avg_temp
df_dummed['var_max_temp'] = df_dummed['var_max_temp'].apply(lambda x: ((x**2)/2)**0.5)

# Desviacion tipica Temp min
df_dummed['var_min_temp'] = df_dummed.min_temp - df_dummed.avg_temp
df_dummed['var_min_temp'] = df_dummed['var_min_temp'].apply(lambda x: ((x**2)/2)**0.5)

# Desviacion tipica viento max
df_dummed['var_max_wind'] = df_dummed.max_wind_speed - df_dummed.avg_wind_speed
df_dummed['var_max_wind'] = df_dummed['var_max_wind'].apply(lambda x: ((x**2)/2)**0.5)

# Desviacion tipica viento min
df_dummed['var_min_wind'] = df_dummed.min_wind_speed - df_dummed.avg_wind_speed
df_dummed['var_min_wind'] = df_dummed['var_min_wind'].apply(lambda x: ((x**2)/2)**0.5)

In [186]:
#Averiguar por qué en x_test.csv no tenemos el mismo shape que en train
train_not_test= df_dummed.columns.difference(dftest.columns)
train_not_test

Index(['EPRTRAnnexIMainActivityLabel_Chemical installations for the production on an industrial scale of basic organic chemicals: Phosphorus-containing hydrocarbons', 'pollutant'], dtype='object')

In [190]:
df_dummed.drop('EPRTRAnnexIMainActivityLabel_Chemical installations for the production on an industrial scale of basic organic chemicals: Phosphorus-containing hydrocarbons',axis = 1,inplace = True)

In [195]:
dftest['pollutant']= clf.predict(dftest[dftest.columns[1:]])

In [202]:
dftest[['test_index','pollutant']].to_csv('predictions.csv')

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)
{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800]}

''' # Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)'''

# Grid search utilizado para saber cuales eran los mejores parametros.... tarda muchisimo en ejecutar y no gana tanta precision,
#{'n_estimators': 1600, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': 90, 'bootstrap': True}

In [ ]:
# lectura pdfs 
def leer_pdf(ruta):

    from pdfminer.high_level import extract_text
    s = extract_text(ruta)
    s = s.split('\n')

    ls_pdf = []
    for i in s:
        if i == '':
            continue
        else:
            ls_pdf.append(i)

    ls_def=[]
    ls_def = ls_pdf[15:-4]
    ls_new = [ls_def[1], ls_def[7], ls_def[15], ls_def[17], ls_def[19], ls_def[21], ls_def[22], ls_def[24]]
    ls_new[2] = ls_new[2][16:]
    ls_new.append(ls_new[7])
    ls_new[7] = ls_new[6]
    ls_new[6] = ls_new[5][:8]
    ls_new[5]= ls_new[4]
    ls_new[4]= ls_new[3][25:]
    ls_new[3] = ls_new[3][:8]
    return ls_new
import os 
carpeta = os.listdir("train6")
ls_dire = []

for i in carpeta:
    i = "train6\" + i
    ls_dire += [i]

ls_noentran = []
ls_final = []
for j in ls_dire:
    try:
        ls_final.append(leer_pdf(j))
    except:
        ls_noentran.append(i)

In [40]:
'''#Varianza temp maxima
df_dummed['var_max_temp'] = df_dummed.max_temp - df_dummed.avg_temp
df_dummed['var_max_temp'] = df_dummed['var_max_temp'].apply(lambda x: (x**2)/2)

# Varianza Temp min
df_dummed['var_min_temp'] = df_dummed.min_temp - df_dummed.avg_temp
df_dummed['var_min_temp'] = df_dummed['var_min_temp'].apply(lambda x: (x**2)/2)

#Varianza viento max
df_dummed['var_max_wind'] = df_dummed.max_wind_speed - df_dummed.avg_wind_speed
df_dummed['var_max_wind'] = df_dummed['var_max_wind'].apply(lambda x: (x**2)/2)

#Varianza viento min
df_dummed['var_min_wind'] = df_dummed.min_wind_speed - df_dummed.avg_wind_speed
df_dummed['var_min_wind'] = df_dummed['var_min_wind'].apply(lambda x: (x**2)/2)'''

11206    2.0
2393     0.0
6098     0.0
13933    2.0
2257     0.0
        ... 
25683    1.0
5102     0.0
9529     0.0
4832     1.0
23933    2.0
Name: pollutant, Length: 9845, dtype: float64

76

0        0
1        0
2        0
3        0
4        0
        ..
24475    0
24476    0
24477    0
24478    0
24479    0
Name: EPRTRAnnexIMainActivityLabel_Chemical installations for the production on an industrial scale of basic organic chemicals: Dyes and pigments, Length: 24480, dtype: uint8